## Scraping StreetEasy.com to analyze housing price in New York City 

My goal here is to collect housing prices for both rental and sale in New York city. I looked at three major real estate website including Trulia, Zillow, and StreetEasy. Comparing to the other two websites, StreetEasy gives the most information on the searching results page and the format of each listing is very consistent, which is great for the purpose of web-scraping.<br\ >
<a href="http://streeteasy.com/">
<img "StreetEasy" src="map/streetEasy_logo.jpg" height="30px" width="150px"/></a><br\ >

Web scraping is done using the beautifulsoup package in Python. I created two functions that can loop through all the pages of searching results, and also empty strings to store results. Below are the steps I took to scrape StreetEasy:
1. Analyzing the HTML page: HTML code of a web page can be viewed by right click and selecting 'Inspect'. This helps us identifying the HTML tags of the information to be scraped
2. Making the soup!: It is important to select the correct parser for your data type. I used HTML parser.
3. Navigating the parse tree and iterate through tags: once the soup is made, we have the HTML code in Python. We can then find our desired information by searching through HTML tags.

In [47]:
import os
from bs4 import BeautifulSoup
import pandas as pd

""""
#for file in dir:
#    print(file)
    #with open(file) as f:
        #soup = BeautifulSoup(f, 'html.parser')
            for item in soup.find_all('div', class_="list-card-info"):
                street = item.find('address', class_="list-card-addr").text
                price = item.find('div', class_="list-card-price").text
                where = item.find('div', class_="list-card-details").text
                bed = item.find('ul', class_="list-card-details").text
                bath = item.find('ul', class_="list-card-details").text
                size = item.find('ul', class_="list-card-details").text
                furnished = item.find('ul', class_="list-card-details").text

                print(street, price, where, bed, bath, size, furnished)
"""
"""
# Find all elements with a specific attribute
soup.find_all(attrs={'data-test': 'property-card-price'})

# Find all <a> tags with a specific attribute
soup.find_all('a', attrs={'data-test': 'property-card-link'})

# Find all elements with multiple specific attributes
soup.find_all(attrs={'data-test': 'property-card-price', 'class': 'some-class'})

# Find all <span> tags with a specific attribute and class
soup.find_all('span', attrs={'data-test': 'some-data-test', 'class': 'some-class'})
"""
#print(os.listdir())
import os
from bs4 import BeautifulSoup

# Create an empty list to store data
data = []

path = "Pretty_Resources/Zillow/Zillow Rentals"
for file in os.listdir(path):
    if file.endswith(".html"):
        file_path = os.path.join(path, file)  # Construct the full file path
        with open(file_path) as f:
            soup = BeautifulSoup(f, 'html.parser')

            # Find all <address> tags with data-test="property-card-addr"
            for address_tag in soup.find_all('address', attrs={'data-test': 'property-card-addr'}):
                address = address_tag.get_text(strip=True)


            for price_tag in soup.find_all('span', class_='PropertyCardWrapper__StyledPriceLine-srp__sc-16e8gqd-1 iMKTKr',
                                          attrs={'data-test': 'property-card-price'}):
                price = price_tag.get_text(strip=True)


            # Find all <a> tags with class="StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jnnxAW property-card-link"
            # and data-test="property-card-link"
            for a_tag in soup.find_all('a', class_='StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0',
                                       attrs={'data-test': 'property-card-link'}):
                url = a_tag.get('href')
                address_inside_a = a_tag.text.strip()

            # Find all <ul> tags with class="StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0"
            for ul_tag in soup.find_all('ul', class_='StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0'):
                # Find all <li> tags within the <ul> tag
                home_details = ul_tag.find_all('li')
                # Initialize variables to store bedroom, bathroom, and square footage information
                bedrooms = None
                bathrooms = None
                sqft = None
                # Extract data from each <li> tag
                for li_tag in home_details:
                    text = li_tag.get_text(strip=True)
                    if 'bd' in text:
                        bedrooms = text.replace('bd', '').strip()
                    elif 'ba' in text:
                        bathrooms = text.replace('ba', '').strip()
                    elif 'sqft' in text:
                        sqft = text.replace('sqft', '').strip()
                # Append data for each property to the list
                data.append({'Address': address, 'Price': price, 'URL': url, 'Bedrooms': bedrooms,
                             'Bathrooms': bathrooms, 'Square Footage': sqft})


# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

                                      Address      Price  \
0    1092 Lenox Rd APT 2R, Brooklyn, NY 11212  $2,800/mo   
1    1092 Lenox Rd APT 2R, Brooklyn, NY 11212  $2,800/mo   
2    1092 Lenox Rd APT 2R, Brooklyn, NY 11212  $2,800/mo   
3    1092 Lenox Rd APT 2R, Brooklyn, NY 11212  $2,800/mo   
4    1092 Lenox Rd APT 2R, Brooklyn, NY 11212  $2,800/mo   
..                                        ...        ...   
162   45 161st St FLOOR 2, Flushing, NY 11358  $2,800/mo   
163   45 161st St FLOOR 2, Flushing, NY 11358  $2,800/mo   
164   45 161st St FLOOR 2, Flushing, NY 11358  $2,800/mo   
165   45 161st St FLOOR 2, Flushing, NY 11358  $2,800/mo   
166   45 161st St FLOOR 2, Flushing, NY 11358  $2,800/mo   

                                                   URL    Bedrooms Bathrooms  \
0    https://www.zillow.com/homedetails/1092-Lenox-...  $2,756+2 s      None   
1    https://www.zillow.com/homedetails/1092-Lenox-...          3s         1   
2    https://www.zillow.com/homedetails

Scraping Pages:   0%|          | 0/1411 [00:00<?, ?it/s]

Request failed: 403 Client Error: Forbidden for url: https://streeteasy.com/for-rent/nyc?page=1
done


,street,price,where,bed,bath,size,furnished


## Data Manipulation

For some listings the information on number of bedroom, number of bathroom, and apartment size is incomplete or mixed up. I performed data manipulation to fix the mistaken values and clean up the extra symbols such as comma and dollar sign. <br\ >
Finally, I have two data sets containing the housing information for apartments for rent and apartments for sale. My for sale data set has 8,456 rows and 8 columns, and the for rent data set has 20,988 rows and 7 columns

NameError: name 'street' is not defined